In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [17]:
train = pd.read_csv('data/train.csv', names=list(map(str, range(56))))

In [18]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,447095,3452,111,22,433,214,3677,252,210,74,...,0,0,0,0,0,0,0,1,0,0
1,113427,3093,95,9,124,7,4115,234,227,124,...,0,0,0,0,0,0,0,0,0,1
2,66435,2551,61,17,90,5,726,231,202,98,...,0,0,0,0,0,0,0,0,0,1
3,8957,2944,135,3,430,13,1868,224,238,149,...,0,0,0,0,0,0,0,0,0,1
4,434631,3030,327,34,277,101,1973,120,181,190,...,0,1,0,0,0,0,0,0,0,1


In [21]:
test = pd.read_csv('data/test.csv', names=list(map(str, range(55))))

In [22]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,258918,2352,293,23,256,45,201,151,228,215,...,0,0,0,0,0,0,0,0,0,0
1,258134,2790,88,8,330,35,1400,232,227,128,...,0,0,0,0,0,0,0,0,0,0
2,165265,2826,115,7,162,24,1020,232,233,134,...,0,0,0,0,0,0,0,0,0,0
3,522141,3234,137,16,150,20,968,243,232,112,...,0,0,0,0,0,0,0,0,0,0
4,569662,2810,58,22,124,25,3293,230,188,80,...,0,0,0,0,0,0,0,0,0,0


In [49]:
len(test)

174303

In [23]:
cols = list(train.columns)
len(cols)

56

In [24]:
features_cols = cols[1:-1]

In [26]:
X = train[features_cols]
y = train['55']

In [29]:
X.shape
y.shape

(406709,)

In [30]:
train_x, val_x, train_y, val_y = train_test_split(X, y, random_state=45, test_size=0.2)

In [31]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

(325367, 54)
(325367,)
(81342, 54)
(81342,)


In [32]:
logreg = LogisticRegression()

In [33]:
logreg.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
yhat = logreg.predict_proba(val_x)

In [36]:
accuracy_score(yhat, val_y)

0.7559071574340439

In [37]:
roc_auc_score(yhat, val_y)

0.7560909872555207

In [42]:
test_feats = test[features_cols]

In [44]:
test_y = logreg.predict_proba(test_feats)

In [46]:
zero_high = test_y[:, 0]

In [57]:
one_high = test_y[:, 1]

In [48]:
len(zero_high)

174303

In [50]:
ids = test['0']

In [51]:
test_pred_0 = pd.Series(zero_high)

In [58]:
test_pred_1 = pd.Series(one_high)

In [59]:
df = pd.DataFrame.from_items([('key', ids), ('score', test_pred_1)])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  """Entry point for launching an IPython kernel.


In [60]:
df.head()

,key,score
0,258918,0.105929
1,258134,0.944964
2,165265,0.795335
3,522141,0.170941
4,569662,0.369707


In [61]:
df.to_csv('Experiment-01-col1.csv', index=False)

In [64]:
xgbc = XGBClassifier()

In [65]:
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [68]:
yhat = xgbc.predict(val_x)

In [69]:
accuracy_score(yhat, val_y)

0.7839738388532369

In [70]:
roc_auc_score(yhat, val_y)

0.7858537841580313

In [72]:
test_y = xgbc.predict_proba(test_feats)

In [73]:
test_y

array([[0.888552  , 0.11144797],
       [0.23412728, 0.7658727 ],
       [0.1874916 , 0.8125084 ],
       ...,
       [0.45576787, 0.54423213],
       [0.23982805, 0.76017195],
       [0.79518723, 0.20481277]], dtype=float32)

In [74]:
xgbc_pred_1 = pd.Series(test_y[:, 1])

In [75]:
df = pd.DataFrame.from_items([('key', ids), ('score', xgbc_pred_1)])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  """Entry point for launching an IPython kernel.


In [76]:
df.head()

,key,score
0,258918,0.111448
1,258134,0.765873
2,165265,0.812508
3,522141,0.256052
4,569662,0.706501


In [78]:
df.to_csv('Experiment-02-col1.csv', index=False)